In [1]:
import pandas as pd
import datetime as dt
from datetime import timedelta
import time
import dateutil
from dateutil.parser import parse
import logging
import importlib
importlib.reload(logging)

<module 'logging' from '/anaconda3/lib/python3.7/logging/__init__.py'>

In [2]:
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

In [3]:
logging.info("now starting...")

12:44:00 INFO:now starting...


In [4]:
logging.info("...before reading csv file.")
# read a csv into a dataframe
df = pd.read_csv('data/IMPORT_CONTROL.txt')
logging.info("...after reading csv file.")

df.head()

12:44:01 INFO:...before reading csv file.
12:44:11 INFO:...after reading csv file.


,AGENCY,EXIT_STATUS_CD,EXIT_STATUS_TEXT,IMPORT2_END_TS,IMPORT2_START_TS,PROCESS_END_TS,PROCESS_START_TS,SITE_ID,TOTAL_RECORDS
0,25,0.0,NaN,NaN,NaN,9/13/2018 12:00:02 AM,9/13/2018 12:00:01 AM,10000,6
1,25,0.0,NaN,NaN,NaN,9/13/2018 12:00:02 AM,9/13/2018 12:00:01 AM,1000,6
2,75,0.0,NaN,NaN,NaN,9/13/2018 12:00:03 AM,9/13/2018 12:00:02 AM,23005,2
3,141,0.0,NaN,NaN,NaN,9/13/2018 12:00:03 AM,9/13/2018 12:00:03 AM,14004,2
4,37,0.0,NaN,NaN,NaN,9/13/2018 12:00:06 AM,9/13/2018 12:00:05 AM,25000,178


In [ ]:
logging.info("...before creating new column.")
# need to make a field to uniquely identify each agency
# concatenating the SITE_ID and the AGENCY will do this
named = df.assign(uniquename = df.SITE_ID.astype(str) + '-' + df.AGENCY.astype(str))
logging.info("...after creating new column.")

# new dataframe to record new info
# interval_df = pd.DataFrame(columns=['uniquename', 'timestamp', 'interval', 'origindex'])

In [ ]:
# use the groupby functions to look at events for each agency
logging.info("...before groupby.")
grouped = named.groupby('uniquename')
logging.info("...after groupby.")

In [ ]:
logging.info("...before dict")
# make a dictionary of the group names
groups = dict(list(grouped))
logging.info("...after dict")


In [ ]:
data_rows = []
i = 0
# for each group
for name in groups:
    # print(name)
    # get the group as a dataframe
    g = grouped.get_group(name)
    # logging.info("length of group: {}".format(len(g)))
    j = 0
    t = time.process_time()
    # then go through the dataframe row by row
    for r in g.itertuples():
        # logging.info("..on row# {}".format(r))
        # only for rows > 0
        row = {}
        if j > 0 and isinstance(r.PROCESS_END_TS, str):
            # calculate seconds between this timestamp and the timestamp from the last row
            thistime = parse(r.PROCESS_END_TS)
            lasttime = parse(last)
            secs = (thistime - lasttime).total_seconds()
            # print(r.Index, thistime, lasttime, secs)
            # interval_df = interval_df.append({'timestamp': thistime, 'uniquename': name, 'interval': secs, 'origindex': r.Index}, ignore_index=True)
            # set the dictionary with the values from this calculated interval
            row['timestamp'] = thistime
            row['uniquename'] = name
            row['interval'] = secs
            row['origindex'] = r.Index
            last = r.PROCESS_END_TS
        if j == 0 and isinstance(r.PROCESS_END_TS, str):
            last = r.PROCESS_END_TS
        j += 1
        # append the dictionary to the data_rows array
        data_rows.append(row)
    i += 1
    dur = time.process_time() - t
#    logging.info("dur: {}, num: {}, avg: {}".format(dur, len(g), dur / len(g)))
    if i > 200000:
        break
    if i == 1 or i%20 == 0:
        logging.info("agency #{} out of {}".format(i, len(groups)))

# create the dataframe from the data_rows array
interval_df = pd.DataFrame(data_rows)
# then write it out to a file
interval_df.to_csv('data/intervals.csv')